<a href="https://colab.research.google.com/github/misrori/goldhand_apps/blob/main/dash_crypto_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install goldhand dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.7 MB/s eta 0:00:00
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=07c13bb7b26baebd37616f61d04e92ad1ab8712866396388af22b268a7c67ff5
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


# automatic

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import time
from goldhand import *

tw = Tw()
tw.crypto.loc[tw.crypto['crypto_common_categories'].isna(), 'crypto_common_categories']= ''

category = 'layer-1' # @param ["layer-1", "smart-contract-platforms", "memecoins", "centralized-exchange", "cryptocurrencies", "sec-security-token", "enterprise-solutions", "payments", "scaling", "oracles", "data-management-ai", "web3", "privacy", "distributed-computing-storage", "collectibles-nfts", "lending-borrowing", "interoperability", "defi", "dao", "decentralized-exchange", "derivatives", "developments-tools", "gaming", "marketplace", "", "internet-of-things", "logistics", "metaverse", "sports", "social-media-content", "algorithmic-stablecoins", "seigniorage", "loyalty-rewards", "asset-backed-tokens", "prediction-markets", "move-to-earn", "energy", "identity", "marketing", "fundraising", "wrapped-tokens", "jobs", "education", "asset-management", "analytics", "e-commerce"]
df = tw.crypto.loc[tw.crypto['crypto_common_categories'].apply(lambda x: category in x) ]
ndays =1000 # @param {type:"slider", min:100, max:1000, step:10}
seconds = 10 # @param {type:"slider", min:5, max:20, step:1}


name_list = list(zip(list(df['base_currency_desc']), list(df['ticker'])))



# Dash app
app = dash.Dash(__name__)
app.title = 'Goldhand'

app.layout = html.Div([
    dcc.Graph(id='live-update-graph', config={'scrollZoom': False}),

    dcc.Interval(
        id='interval-component',
        interval=seconds*1000,  # in milliseconds
        n_intervals=0
    )
],style={ 'margin': 10, 'gap': 10, 'height':'100vh'}
)


@app.callback(
    Output('live-update-graph', 'figure'),
    Input('interval-component', 'n_intervals')
)
def update_graph(n):
    # Get the next name in the list
    ticker = name_list[n % len(name_list)][1]

    # Get the plot for the next name
    t = GoldHand(ticker)
    fig = t.plotly_last_year(plot_title=tw.get_plotly_title(ticker), plot_height=1000, ndays=ndays)


    return fig

if __name__ == '__main__':

    app.run_server(debug=True, jupyter_mode='external', title= 'Goldhand')




Dash app running on:


<IPython.core.display.Javascript object>

# manual

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
from goldhand import *


tw = Tw()
tw.crypto.loc[tw.crypto['crypto_common_categories'].isna(), 'crypto_common_categories']= ''

category = 'layer-1' # @param ["layer-1", "smart-contract-platforms", "memecoins", "centralized-exchange", "cryptocurrencies", "sec-security-token", "enterprise-solutions", "payments", "scaling", "oracles", "data-management-ai", "web3", "privacy", "distributed-computing-storage", "collectibles-nfts", "lending-borrowing", "interoperability", "defi", "dao", "decentralized-exchange", "derivatives", "developments-tools", "gaming", "marketplace", "", "internet-of-things", "logistics", "metaverse", "sports", "social-media-content", "algorithmic-stablecoins", "seigniorage", "loyalty-rewards", "asset-backed-tokens", "prediction-markets", "move-to-earn", "energy", "identity", "marketing", "fundraising", "wrapped-tokens", "jobs", "education", "asset-management", "analytics", "e-commerce"]

df = tw.crypto.loc[tw.crypto['crypto_common_categories'].apply(lambda x: category in x) ]


name_list = list(zip(list(df['base_currency_desc']), list(df['ticker'])))

# Dash app
app = dash.Dash(__name__)
app.title = 'Goldhand'

app.layout = html.Div([
    dcc.Dropdown(
        id='name-dropdown',
        options=[{'label': x[0], 'value': x[1]} for x in name_list],
        value=name_list[0][1]  # Set the default selected value
    ),
    html.Button('Previous', id='prev-button', n_clicks=0),
    html.Button('Next', id='next-button', n_clicks=0),
    dcc.Input(
        id='ndays-input',
        type='number',
        value=530,  # Set the default value
        step=10
    ),
    dcc.Graph(id='live-update-graph'),

])

@app.callback(
    [Output('live-update-graph', 'figure'),
     Output('name-dropdown', 'value')],
    [Input('name-dropdown', 'value'),
     Input('next-button', 'n_clicks'),
     Input('prev-button', 'n_clicks'),
     Input('ndays-input', 'value')]
)
def update_graph(selected_name, next_clicks, prev_clicks, ndays):
    # Determine which button was clicked
    ctx = dash.callback_context
    triggered_id = ctx.triggered_id

    # Get the current name index
    current_index = name_list.index((df.loc[df['ticker'] == selected_name, 'base_currency_desc'].values[0], selected_name))

    # Handle button clicks
    if triggered_id == 'next-button':
        next_index = current_index + 1
    elif triggered_id == 'prev-button':
        next_index = current_index - 1
    else:
        next_index = current_index

    # Ensure the index is within the valid range
    next_index = next_index % len(name_list)

    # Get the next name
    next_name = name_list[next_index][1]

    # Get the plot for the next name
    t = GoldHand(next_name)
    fig = t.plotly_last_year(plot_title=tw.get_plotly_title(next_name), plot_height=1000, ndays=ndays)

    return fig, next_name

if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode='external')


Dash app running on:


<IPython.core.display.Javascript object>